In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
from constantes import WINSIG_SERVICIO_FARMACIA_CC_SIGCOM

In [38]:
def desglosar_centro_de_costo(desglose, total_dinero):
    con_dinero = desglose.copy()
    con_dinero['TOTAL_X_PORCENTAJE'] = con_dinero['PORCENTAJES'] * total_dinero

    return con_dinero


def rellenar_columna_medicamentos_winsig():
    df_winsig = pd.read_excel('input\\Informe Winsig Septiembre 2022.xlsx', header=7)
    df_winsig = df_winsig.dropna(axis=1, how='all').dropna(axis=0, how='all')
    archivo_produccion = pd.ExcelFile('input\\output_producciones.xlsx')

    desglose_pabellon = pd.read_excel(archivo_produccion, sheet_name='PABELLÓN')
    total_pabellon = df_winsig.query('SERVICIO == "B. PABELLON"')['Gasto Servicio'].iloc[0]
    con_gastos_pabellon = desglosar_centro_de_costo(desglose_pabellon, total_pabellon)

    desglose_policlinico = pd.read_excel(
        archivo_produccion, sheet_name='CONSULTAS SIN MANEJO DEL DOLOR')
    total_policlinico = df_winsig.query('SERVICIO == "POLICLÍNICO"')['Gasto Servicio'].iloc[0]
    con_gastos_policlinico = desglosar_centro_de_costo(desglose_policlinico, total_policlinico)

    con_gastos_pabellon_para_concatenar = con_gastos_pabellon.iloc[:, [0, 1, 2, -1, 3]]
    con_gastos_pabellon_para_concatenar.iloc[:, [1, 2, -1]] = None
    con_gastos_pabellon_para_concatenar.columns = df_winsig.columns

    con_gastos_policlinico_para_concatenar = con_gastos_policlinico.iloc[:, [0, 1, 2, -1, 3]]
    con_gastos_policlinico_para_concatenar.iloc[:, [1, 2, -1]] = None
    con_gastos_policlinico_para_concatenar.columns = df_winsig.columns

    winsig_concatenado = df_winsig.copy()
    winsig_mas_desglose = pd.concat([winsig_concatenado, con_gastos_pabellon_para_concatenar])
    winsig_mas_desglose = pd.concat([winsig_mas_desglose, con_gastos_policlinico_para_concatenar])
    servicios_a_sacar = ['B. PABELLON', 'PABELLÓN', 'POLICLÍNICO', 'CONSULTAS SIN MANEJO DEL DOLOR']
    servicios_a_dejar = ~(winsig_mas_desglose['SERVICIO'].isin(servicios_a_sacar))

    winsig_final = winsig_mas_desglose[servicios_a_dejar]
    winsig_sigcom = winsig_final.copy()
    winsig_sigcom['CC_SIGCOM'] = winsig_sigcom['SERVICIO'].apply(
        lambda x: WINSIG_SERVICIO_FARMACIA_CC_SIGCOM[x])

    agrupado_sigcom = winsig_sigcom.groupby('CC_SIGCOM')['Gasto Servicio'].sum()
    formato = pd.read_excel('input\\Formato 4_Distribución Suministro 2022-10.xlsx')
    formato = formato.set_index('Centro de Costo')

    for cc, valor in agrupado_sigcom.items():
        print(f'Imputando {cc} {valor}')
        formato.loc[cc, '30-MEDICAMENTOS'] = valor

    return formato


In [39]:
formato_relleno = rellenar_columna_medicamentos_winsig()

Imputando 102-HOSPITALIZACIÓN CARDIOVASCULAR 0.0
Imputando 15008-CONSULTA NUTRICIÓN 900679.5823545479
Imputando 15010-CONSULTA OTROS PROFESIONALES 85764711.34198308
Imputando 15026-PROCEDIMIENTOS DE CARDIOLOGÍA 113294.0
Imputando 15105-CONSULTA CARDIOLOGÍA 32424464.964763723
Imputando 15107-CONSULTA ONCOLOGÍA 18313818.174542475
Imputando 15111-CONSULTA NEUMOLOGÍA 169527912.50095603
Imputando 15123-PROGRAMA MANEJO DEL DOLOR 5494747.0
Imputando 15201-CONSULTA CIRUGÍA GENERAL 29222048.671947554
Imputando 15220-CONSULTA CIRUGIA CARDIACA 30222803.76345261
Imputando 166-UNIDAD DE CUIDADOS INTENSIVOS 42754923.0
Imputando 195-UNIDAD DE TRATAMIENTO INTENSIVO ADULTO 38023316.0
Imputando 253-PROCEDIMIENTOS DE HEMODINAMIA 1930656.0
Imputando 267-PROCEDIMIENTOS ENDOSCÓPICOS 105884.0
Imputando 464-QUIRÓFANOS CARDIOVASCULAR 52576108.666666664
Imputando 484-QUIRÓFANOS TORACICA 26288054.333333332
Imputando 66-HOSPITALIZACIÓN MEDICINA INTERNA 72548916.0
